# 决策树

**决策树的定义** 
分类决策树模型是一种描述对实例进行分类的树形结构。决策树由结点（node）和有向边（directed edge）组成。结点有两种类型：内部结点（internal node）和叶结点（leaf node）。内部结点表示一个特征或属性（features），叶结点表示一个类（labels）。

用决策树对需要测试的实例进行分类：从根节点开始，对实例的某一特征进行测试，根据测试结果，将实例分配到其子结点；这时，每个子结点对应着该特征的一个取值。如此递归地对实例进行测试并分配，直至达到叶结点。最后将实例分配到叶结点的类中。

**信息熵 & 信息增益**
熵（entropy）: 熵指的是体系的混乱的程度，在不同的学科中也有引申出的更为具体的定义，是各领域十分重要的参量。

信息论（information theory）中的熵（香农熵）: 是一种信息的度量方式，表示信息的混乱程度，也就是说: 信息越有序，信息熵越低。例如: 火柴有序放在火柴盒里，熵值很低，相反，熵值很高。

信息增益（information gain）: 在划分数据集前后信息发生的变化称为信息增益。

**决策树工作原理**
如何构造一个决策树？以下为伪代码：
```
def createBranch():
    检测数据集中的所有数据的分类标签是否相同
    If so return 类标签
    Else:
        寻找划分数据集的最好特征（划分之后信息熵最小，也就是信息增益最大的特征）
        划分数据集
        创建分支节点
            for 每个划分的子集
                调用函数 createBranch() （创建分支的函数）并增加返回结果到分支节点中
        return 分支节点
```

**决策树开发流程**
1. 收集数据: 可以使用任何方法。
2. 准备数据: 树构造算法 (这里使用的是ID3算法，只适用于标称型数据，这就是为什么数值型数据必须离散化。 还有其他的树构造算法，比如CART)
3. 分析数据: 可以使用任何方法，构造树完成之后，我们应该检查图形是否符合预期。
4. 训练算法: 构造树的数据结构。
5. 测试算法: 使用训练好的树计算错误率。
6. 使用算法: 此步骤可以适用于任何监督学习任务，而使用决策树可以更好地理解数据的内在含义。

**项目案例：判定鱼类和非鱼类**

项目描述：根据以下2个特征，将动物分成两类：鱼类和非鱼类。
特征：
1. 不浮出水面是否可以生存
2. 是否有脚蹼

1. 收集数据集，这里从程序直接给出

In [1]:

'''
@description: 创建数据集
@param {None} 
@return: 返回数据集和对应的label标签
'''
def creatDataSet():
    '''
    |不浮出水面可以生存| 是否有脚蹼 |属于鱼类
    |1-是----------------------|-是------------|是
    |2-是--------------------- |-是------------|是
    |3-是--------------------- |-否------------|否
    |4-否--------------------- |-是------------|否
    |5-否--------------------- |-是------------|否
    no surfacing:不露出水面是否可以生成？
    flippers:是否有脚蹼？
    '''
    dataSet = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no']
    ]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels



2. 计算数据集的信息熵

熵定义为信息的期望值，在明晰这个概念之前，必须知道信息的定义。如果待分类的事务可能划分在多个分类之中，则符号 $x_i$ 的信息定义为 $$ l(x_i) = -log_2p(x_i) $$

其中 $ p(x_i) $ 是选择该分类的概率。
为了计算信息熵，我们需要计算所有类别的所有可能值包含的信息期望值，通过下面的公式得到：
$$ H = -\displaystyle\sum_{i=1}^n p(x_i)log_2 p(x_i) $$
公式中的 $n$ 是分类的个数

In [ ]:

'''
@description: 计算给定数据集的香农熵
@param {type} 数据集
@return: 返回 每一组feature下的某个分类下，香农熵的信息期望
eg:dataSet下 Yes/No 分类下香农熵大小 0.970950594455 信息越有序，信息熵越低。
'''
def calcShannonEnt(dataSet):
    # -----------计算香农熵的第一种实现方式start------------------------------------
    #
    # numEntries = len(dataSet)
    # labelsCounts = {}
    # for featVec in dataSet:
    #     currentLabel = featVec[-1]
    #     if currentLabel not in labelsCounts.keys():
    #         labelsCounts[currentLabel] = 0
    #     labelsCounts[currentLabel] += 1
    # shannonEnt = 0.0
    # for key in labelsCounts:
    #     prob = float(labelsCounts[key])/numEntries
    #     shannonEnt -= prob * log(prob, 2)
    # -----------计算香农熵的第一种实现方式end----------------------------------------

    # # -----------计算香农熵的第二种实现方式start------------------------------------
    # # 统计标签出现的次数
    #eg:Counter({'no': 3, 'yes': 2})
    label_count = Counter(data[-1] for data in dataSet)
    # # 计算概率
    probs = [float(p[1]) / len(dataSet) for p in label_count.items()]
    # # 计算香农熵
    shannonEnt = sum([-p * log(p, 2) for p in probs])
    # -----------计算香农熵的第二种实现方式end----------------------------------------
    return shannonEnt


'''
@description: 按照给定特征划分数据集
@param {dataSet 数据集 : 待划分的数据集 
        index 表示每一行的index列 :划分数据集的特征
        value 表示index列对应的value值 :需要返回的特征的值
} 
@return: index列为value的数据集【该数据集需要排除index列】
demo:dataSet=[[1, 1, 'yes'],
      [1, 1, 'yes'],
      [1, 0, 'no'],
      [0, 1, 'no'],
      [0, 1, 'no']]
      splitDataSet(dataSet,0,1)
      这个方法代表的意思就是在上面的数据集中找到第0列值是1的数据 返回出来 并且返回值中不包含第0列的值
      现在可以看到第0列值为1的数据有[1, 1, 'yes']，[1, 1, 'yes']，[1, 0, 'no']三个，然后去掉第0列
      的值返回出来的就是[1, 'yes'], [1, 'yes'], [0, 'no']
'''
def splitDataSet(dataSet, index, value):
    # -----------划分数据集的第一种方式 start------------------------------------
    retDataSet = []
    for featVec in dataSet:
        # index列为value的数据集【该数据集需要排除index列】
        # 判断index列的值是否为value
        if featVec[index] == value:
            # [:index]表示前index行，即若 index 为2，就是取 featVec 的前 index 行
            reduceFeatVec = featVec[:index]
            # [index+1:]表示从跳过 index 的 index+1行，取接下来的数据
            # 收集结果值 index列为value的行【该行需要排除index列】
            '''
            对于extend append
            list.append(object) 向列表中添加一个对象object
            list.extend(sequence) 把一个序列seq的内容添加到列表中
            1、使用append的时候，是将new_media看作一个对象，整体打包添加到music_media对象中。
            2、使用extend的时候，是将new_media看作一个序列，将这个序列和music_media序列合并，并放在其后面。
            result = []
            result.extend([1,2,3])
            print result
            result.append([4,5,6])
            print result
            result.extend([7,8,9])
            print result
            结果：
            [1, 2, 3]
            [1, 2, 3, [4, 5, 6]]
            [1, 2, 3, [4, 5, 6], 7, 8, 9]
            '''
            reduceFeatVec.extend(featVec[index+1:])
            retDataSet.append(reduceFeatVec)
    # -----------划分数据集的第一种方式 end------------------------------------
    # -----------划分数据集的第二种方式 start------------------------------------
    #retDataSet = [data for data in dataSet for i,v in enumerate(data) if i == index and v == value]
    #这个没有排除第index列
    return retDataSet

'''
@description: 选择最好的数据集划分方式
@param {dataSet:数据集} 
@return: bestFeature ：最优的特征列
demo:可以看出infoGain信息增益0的时候是比较大的 所以最好的特征是0
输出：infoGain= 0.419973094022 bestFeature= 0 0.970950594455 0.550977500433
    infoGain= 0.170950594455 bestFeature= 1 0.970950594455 0.8
    最后返回的最好的数据集bestFeature=0
'''
def chooseBestFeatureToSplit(dataSet):
    # -----------选择最优特征的第一种方式 start------------------------------------
    #先求第一行有多少特征 最后一行是label标签所以减去1
    numFeatures = len(dataSet[0]) - 1
    #Label标签信息熵
    baseEntropy = calcShannonEnt(dataSet)
    # 最优的信息增益值, 和最优的Featurn编号
    bestInfoGain, bestFeature = 0.0, -1
    for i in range(numFeatures):
        # 获取每一个实例的第i+1个feature，组成list集合
        #eg:[1, 1, 1, 0, 0] [1, 1, 0, 1, 1]
        featList = [example[i] for example in dataSet]
        # 获取剔重后的集合，使用set对list数据进行去重
        #eg:set([0, 1]) set([0, 1])
        uniqueVals = set(featList)
        # 创建一个临时的信息熵
        newEntropy = 0.0
        # 遍历某一列的value集合，计算该列的信息熵 
        # 遍历当前特征中的所有唯一属性值，对每个唯一属性值划分一次数据集，计算数据集的新熵值，并对所有唯一特征值得到的熵求和。
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            #计算这个subDataSet占整个dataSet的概率是多少
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        # gain[信息增益]: 划分数据集前后的信息变化， 获取信息熵最大的值
        # 信息增益是熵的减少或者是数据无序度的减少。最后，比较所有特征中的信息增益，返回最好特征划分的索引值。
        infoGain = baseEntropy - newEntropy
        #print('infoGain=', infoGain, 'bestFeature=', i, baseEntropy, newEntropy)
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    # -----------选择最优特征的第一种方式 end------------------------------------
    # # -----------选择最优特征的第二种方式 start------------------------------------
    # # 计算初始香农熵
    # base_entropy = calcShannonEnt(dataSet)
    # best_info_gain = 0
    # best_feature = -1
    # # 遍历每一个特征
    # for i in range(len(dataSet[0]) - 1):
    #     # 对当前特征进行统计
    #     feature_count = Counter([data[i] for data in dataSet])
    #     # 计算分割后的香农熵
    #     new_entropy = sum(feature[1] / float(len(dataSet)) * calcShannonEnt(splitDataSet(dataSet, i, feature[0])) \
    #                    for feature in feature_count.items())
    #     # 更新值
    #     info_gain = base_entropy - new_entropy
    #     print('No. {0} feature info gain is {1:.3f}'.format(i, info_gain))
    #     if info_gain > best_info_gain:
    #         best_info_gain = info_gain
    #         best_feature = i
    # return best_feature
    # # -----------选择最优特征的第二种方式 end------------------------------------
    return bestFeature



　　loop(dataSet):

　　　　if dataSet 所有分类都是同一个，返回该分类

　　　　if dataSet 没有可以进行选择的条件了，返回该结果集大部分指向的分类

　　　　通过信息增益指标，选择当前dataSet最有决定性的特征

　　　　for 该特征的值：

　　　　　　该值所得出的子集subDataSet

　　　　　　tree = loop(subDataSet) 构建该特征的该值的子树

　　　　return tree
